### Possible improvements:
1) attempted the method as in QInfer (DirectViewParallelizedModels) but not even dill nor cloud_pickle seem to handle appropriately the closures - or maybe some other problem?

...

In [2]:
%matplotlib inline

import qinfer as qi
#import qutip as qt
import numpy as np
import scipy as sp
from IPython.display import display, Math, Latex
from functools import partial
import matplotlib.pyplot as plt
import importlib as imp

import ipyparallel as ipp

In [3]:
import datetime
import sys, os
import time as time

In [4]:
import logging as logging
import sys

%run LoggingCode.py

In [5]:
# tell the user where files will be saved
dire = os.getcwd()
dire = dire+"/tests/T"
dire = os.path.normpath(dire)
print('Files will be saved in:')
print(dire)

Files will be saved in:
C:\Users\ag15141\Local Documents\Cloud\Dropbox (Qsim)\Projects\CU4 chip\QHLandD\MultiPar_GenSim cleanup\tests\T


In [6]:
# Add personalized library to PYTHONPATH
sys.path.append(os.path.join("..","Libraries","QML_lib"))

In [7]:
from Norms import *
from IOfuncts import *
from EvalLoss import *

import ProbeStates as pros
import multiPGH as mpgh
import parGenSim_IQLE as gsi
import Evo as evo

# Parallelize under n cores

In [8]:
from joblib import Parallel, delayed
import multiprocessing


num_cores = multiprocessing.cpu_count()
print('I have found: '+repr(num_cores)+ ' cores')

start = time.clock()
Parallel(n_jobs=8)
end=time.clock()
print('timelapse for initialising parallel: ', end-start)

I have found: 8 cores
timelapse for initialising parallel:  0.000831282439381931


# Results from Parallelized step-by-step running

### parallelization at likelihood stage

In [9]:
imp.reload(pros)
imp.reload(mpgh)
imp.reload(gsi)
imp.reload(evo)

<module 'Evo' from 'C:\\Users\\ag15141\\Local Documents\\Cloud\\Dropbox (Qsim)\\Projects\\CU4 chip\\QHLandD\\MultiPar_GenSim cleanup\\Evo.py'>

In [10]:
probecounter = 0

oplist=np.array([evo.sigmay(), evo.sigmax()])
#oplist=np.array([evo.sigmaz()])
oplist=np.dot(1.0, oplist)
print('Oplist: ' + repr(oplist))

true_params = np.array([[0.3, 0.6]])
#true_params = np.array([[0.7]])
print('Chosen true_params: ' + str(true_params))

#Listing eigenvectors
# eiglist = pros.list_randomeig(oplist)
# eiglist = np.append(eiglist, np.array([1, 0]))
# eiglist = np.reshape(eiglist, [len(oplist)+1,2])
# print('Chosen probestates: ' + repr(eiglist))

#eiglist=sp.absolute(sp.linalg.orth(oplist[0]))
#print('Chosen probestates: ' + repr(eiglist))

# newprobe = list(map(lambda wgts_list: np.sum(wgts_list*eiglist, axis=0), wgts_list))
# newprobe

probestate=pros.choose_probe(oplist,true_params)
print('Chosen probestate: ' + str(probestate))

sigw = 0.3
siga = 0.3

prior = qi.MultivariateNormalDistribution(
     [true_params[0, 0] + sigw/3, true_params[0, 1]-siga/3], [[sigw **2, 0], [0, siga **2]])
# prior = qi.UniformDistribution([0., 1])


model = gsi.parGenSim_IQLE(oplist=oplist, modelparams=true_params, probecounter = 0, probelist= [probestate], solver='scipy', trotter=True)


n_particles = 300
n_experiments = 100

updater = qi.SMCUpdater(model, n_particles, prior, resample_thresh=0.5, resampler = qi.LiuWestResampler(a=0.95), debug_resampling=False)

inv_field = [item[0] for item in model.expparams_dtype[1:] ]
print('Inversion fields are: ' + str(inv_field))
heuristic = mpgh.multiPGH(updater, oplist, inv_field=inv_field)
print('Heuristic output:' + repr(heuristic()))

expparams = np.empty((1, ), dtype=model.expparams_dtype)

print('\nInitialization Ready')

Oplist: array([[[ 0.+0.j,  0.-1.j],
        [ 0.+1.j,  0.+0.j]],

       [[ 0.+0.j,  1.+0.j],
        [ 1.+0.j,  0.+0.j]]])
Chosen true_params: [[ 0.3  0.6]]
Chosen probestate: [ 0.9472136+0.2236068j  0.0527864-0.2236068j]
Inversion fields are: ['w_1', 'w_2']
Heuristic output:array([(1.2084116627328185, 0.8703375649742549, 0.2864076503525312)], 
      dtype=[('t', '<f8'), ('w_1', '<f8'), ('w_2', '<f8')])

Initialization Ready


In [15]:
modelparams = np.tile(np.array([[0.3, 0.6]]),(5000,1))

experiment = heuristic()

start = time.clock()
evolution = evo.pr0fromScipyNC([10], modelparams[:,], np.array([experiment.item(0)[1:]]), oplist, probestate, trotterize=False)
end=time.clock()
print('timelapse as map: ', end-start)
# print(evolution)


start = time.clock()
evolution = np.array([evo.pr0fromScipyNCpp([10], modelparams[i,], np.array([experiment.item(0)[1:]]), oplist, probestate, trotterize=False) 
 for i in range(modelparams.shape[0])])
end=time.clock()
print('timelapse as for cycle: ', end-start)
#print(evolution)


start = time.clock()
evolution = np.array(Parallel(n_jobs=8, backend="threading")
(delayed(evo.pr0fromScipyNCpp)([10], modelparams[i,], np.array([experiment.item(0)[1:]]), oplist, probestate, trotterize=False)
 for i in range(modelparams.shape[0])))
end=time.clock()
print('timelapse as parallel cycle: ', end-start)
#print(evolution)

timelapse as map:  4.100352442566702
timelapse as for cycle:  4.385642219481085
timelapse as parallel cycle:  6.420795104603869


In [61]:
modelparams.shape[0]

10

timelapse:  0.020746637957472558
[array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]]), array([[ 0.24385889]])]


In [50]:
prova = modelparams[1,]
print(prova)
print(len(prova.shape))
prova = np.array([prova])
print(prova)
print(len(prova.shape))

[ 0.3  0.6]
1
[[ 0.3  0.6]]
2


In [111]:
modelparams[:,]

array([[ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6],
       [ 0.3,  0.6]])

In [79]:
from math import sqrt
result = Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in range(300))
#print(result)

In [23]:
eps = np.empty((1,), dtype=updater.model.expparams_dtype)

experiment = heuristic()
print(repr(experiment))

datum = model.simulate_experiment(true_params, experiment)
print(repr(datum))

updater.update(datum, experiment)

array([(1.4473016887027013, 0.5739468683810348, 0.0322945862806962)], 
      dtype=[('t', '<f8'), ('w_1', '<f8'), ('w_2', '<f8')])
1


In [25]:
# counter timestamp to avoid replacement
timestamp = str(datetime.datetime.now()).split('.')[0]
timestamp = "_"+timestamp.replace(" ", "_")
timestamp = timestamp.replace(":", ".")

# probecounter for the choice of the state
probecounter = 0

track_loss = np.empty(n_experiments)
track_cov = np.empty(n_experiments)
track_time = np.empty(n_experiments)

track_particle_locs = np.empty([n_particles, len(true_params[0]), n_experiments])
track_particle_wght = np.empty([n_particles, n_experiments])

start=time.clock()


for idx_experiment in range(n_experiments):
    
    experiment = heuristic()
    #print('Chosen experiment: ' + repr(experiment))
    
    track_time[idx_experiment] = experiment[0][0]
    
    datum = model.simulate_experiment(true_params, experiment)
    #print(repr(datum))
    updater.update(datum, experiment)
    heuristic = mpgh.multiPGH(updater, oplist, inv_field=inv_field)
    
    track_cov[idx_experiment] = np.linalg.norm(updater.est_covariance_mtx())
    
    track_particle_locs[:, :, idx_experiment] = updater.particle_locations
    track_particle_wght[:, idx_experiment] = updater.particle_weights

    new_eval = updater.est_mean()
    
    new_loss = eval_loss(model, new_eval, true_params)
    track_loss[idx_experiment] = new_loss[0]
    
end=time.clock()
print('elapsed time: ' + str(end-start))

elapsed time: 11.060034416615848


# Results from Parallelized qi.perftestmultiple

### parallelization at trial stage

In [56]:
probecounter = 0

oplist=np.array([evo.sigmay(), evo.sigmax()])
#oplist=np.array([evo.sigmaz()])
oplist=np.dot(1.0, oplist)
print('Oplist: ' + repr(oplist))

true_params = np.array([[0.3, 0.6]])
#true_params = np.array([[0.7]])
print('Chosen true_params: ' + str(true_params))

#Listing eigenvectors
# eiglist = pros.list_randomeig(oplist)
# eiglist = np.append(eiglist, np.array([1, 0]))
# eiglist = np.reshape(eiglist, [len(oplist)+1,2])
# print('Chosen probestates: ' + repr(eiglist))

#eiglist=sp.absolute(sp.linalg.orth(oplist[0]))
#print('Chosen probestates: ' + repr(eiglist))

probestate=pros.choose_probe(oplist,true_params)
print('Chosen probestate: ' + str(probestate))

sigw = 0.3
siga = 0.3

prior = qi.MultivariateNormalDistribution(
     [true_params[0, 0] + sigw/3, true_params[0, 1]-siga/3], [[sigw **2, 0], [0, siga **2]])
# prior = qi.UniformDistribution([0., 1])


model = gsi.GenSim_IQLE(oplist=oplist, modelparams=true_params, probecounter = 0, probelist= [probestate], solver='scipy', trotter=True)


"""We do NOT invoke here parallelization"""
par_model = qi.DirectViewParallelizedModel(model, c[:] )

n_particles = 300
n_experiments = 300


extra_updater_args={'resample_thresh': 0.5, 'resampler': qi.LiuWestResampler(a=0.95), 'debug_resampling': True}
updater = qi.SMCUpdater(model, n_particles, prior, **extra_updater_args)

inv_field = [item[0] for item in model.expparams_dtype[1:] ]
print('Inversion fields are: ' + str(inv_field))
heuristic = mpgh.multiPGH(updater, oplist, inv_field=inv_field)
print('Heuristic output:' + repr(heuristic()))

expparams = np.empty((1, ), dtype=model.expparams_dtype)

print('Initialization Ready')

Oplist: array([[[ 0.+0.j,  0.-1.j],
        [ 0.+1.j,  0.+0.j]],

       [[ 0.+0.j,  1.+0.j],
        [ 1.+0.j,  0.+0.j]]])
Chosen true_params: [[ 0.3  0.6]]
Chosen probestate: [ 0.9472136+0.2236068j  0.0527864-0.2236068j]
Inversion fields are: ['w_1', 'w_2']
Heuristic output:array([(1.710067274021547, 0.5432482731598641, 0.44569670572272035)], 
      dtype=[('t', '<f8'), ('w_1', '<f8'), ('w_2', '<f8')])
Initialization Ready


In [58]:
start = time.clock()


perf = qi.perf_test_multiple(model=model, n_trials = 2, n_particles=n_particles, prior=prior,
        n_exp=n_experiments, true_mps = true_params, heuristic_class=partial(mpgh.multiPGH, inv_field=inv_field, t_field='t'), 
                    extra_updater_args = extra_updater_args,
                   apply=lbview.apply)

end=time.clock()
print('elapsed time: ' + str(end-start))

RemoteError: ImportError(No module named 'GenSim_IQLE')